# Phi-4 Mini ONNX Урок за паралелно извикване на функции

Този тетрадка демонстрира как да използвате Phi-4 Mini с ONNX Runtime GenAI за паралелно извикване на функции. Извикването на функции позволява на модела интелигентно да използва външни инструменти и API въз основа на потребителски заявки.

## Преглед

В този урок ще научите как да:
- Настроите Phi-4 Mini с ONNX Runtime GenAI
- Дефинирате схеми на функции за резервиране на полети и хотели
- Използвате насочено генериране с граматика Lark за структурирани изходи
- Изпълнявате паралелни извиквания на функции за сложни сценарии за резервиране на пътувания

## Предварителни изисквания

Преди да стартирате този тетрадка, уверете се, че:
- Сте изтеглили модела Phi-4 Mini ONNX
- Сте инсталирали пакета `onnxruntime-genai`
- Имате основно разбиране за концепциите на извикване на функции


## Стъпка 1: Импортиране на необходимите библиотеки

Първо, ще импортираме необходимите библиотеки за нашата имплементация на извикване на функции.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## Стъпка 2: Настройка и конфигурация на модела

Сега ще конфигурираме модела Phi-4 Mini ONNX. Уверете се, че заменяте пътя с вашата действителна директория на модела.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## Стъпка 3: Конфигурирайте параметрите за генериране

Настройте параметрите за генериране, за да контролирате поведението на изхода на модела. Тези настройки гарантират детерминирани и фокусирани отговори за извикване на функции.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Стъпка 4: Определяне на наличните функции

Тук определяме функциите, които нашият AI асистент може да извиква. В този пример имаме две функции, свързани с пътувания:
1. **booking_flight_tickets**: За резервиране на полети между летища
2. **booking_hotels**: За резервиране на хотелски настанявания

Дефинициите на функциите следват формата на схемата за извикване на функции на OpenAI.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Стъпка 5: Помощни функции за генериране на граматика

Тези помощни функции преобразуват дефинициите на нашите функции във формат на граматика за Lark, който се използва от ONNX Runtime GenAI за насочено генериране. Това гарантира, че моделът извежда валидни извиквания на функции в правилния JSON формат.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## Стъпка 6: Тестване на генерирането на граматика

Нека тестваме нашите функции за генериране на граматика, за да видим как те преобразуват дефинициите на инструментите ни в правилния формат.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## Стъпка 7: Подготовка на системния подканващ текст и генератора

Сега ще създадем системния подканващ текст, който информира модела за наличните инструменти, и ще настроим генератора с параметри за насочено генериране.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## Стъпка 8: Инициализиране на генератора с насочено генериране

Сега ще създадем генератора с нашите конфигурирани параметри и ще приложим граматиката на Lark за насочено генериране.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## Стъпка 9: Тестване на паралелно извикване на функции

Сега нека тестваме нашата настройка със сложен сценарий за резервация на пътуване, който изисква извикване на множество функции.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Генериране на извиквания на функции

Моделът вече ще генерира структурирани извиквания на функции въз основа на потребителската заявка. Благодарение на насоченото генериране, изходът ще бъде във валиден JSON формат, който може да бъде изпълнен директно.

**Очакван резултат**: Моделът трябва да генерира извиквания на функции за:
1. `booking_flight_tickets` с подробности за полет от Пекин (PEK) до Париж (CDG)
2. `booking_hotels` с подробности за настаняване в Париж

Изпълнете клетката по-долу, за да видите генерирането в реално време:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Заключение

🎉 **Поздравления!** Успешно внедрихте паралелно извикване на функции с Phi-4 Mini, използвайки ONNX Runtime GenAI.

### Какво научихте:

1. **Настройка на модела**: Как да конфигурирате Phi-4 Mini с ONNX Runtime GenAI
2. **Дефиниране на функции**: Как да дефинирате схеми за функции за извикване от AI
3. **Ръководено генериране**: Как да използвате граматика Lark за структурирано генериране на изход
4. **Паралелни извиквания на функции**: Как да обработвате сложни заявки, изискващи множество извиквания на функции

### Основни предимства:

- ✅ **Структуриран изход**: Ръководеното генериране гарантира валидни JSON извиквания на функции
- ✅ **Паралелна обработка**: Обработка на множество извиквания на функции в една заявка
- ✅ **Висока производителност**: ONNX Runtime осигурява оптимизирано извеждане
- ✅ **Гъвкава схема**: Лесно добавяне или модифициране на дефиниции на функции

### Ресурси:

- [Phi-4 Mini Документация](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI Документация](https://onnxruntime.ai/docs/genai/)
- [Най-добри практики за извикване на функции](https://platform.openai.com/docs/guides/function-calling)



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматичните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия изходен език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален превод от човек. Ние не носим отговорност за каквито и да е недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
